# Slab model

The purpose of `dustpylib.radtrans.slab` is to implement a simple, plane-parallel slab solution for protoplanetary disks and to compute those based on files from `DustPy` simulations.  

In this notebook we are going to produce a radio images from the [planetary gaps example](https://stammler.github.io/dustpy/example_planetary_gaps.html) in the `DustPy` documentation. This repository contains a reduced `DustPy` output file of the final snapshot of the example model only containing the fields that are required to produce the `RADMC-3D` model. We will use the same to compute such an image with the slab model for comparison.

All quantities are in CGS units.

In the first step we need to load the `DustPy` data file using the `hdf5writer` for this.

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from dustpy import hdf5writer
import dustpy.constants as c
from dustpylib.radtrans import radmc3d

In [ ]:
writer = hdf5writer()
writer.datadir = "example_planetary_gaps"
data = writer.read.output(21)

## Creating the `RADMC-3D` model for comparison

To compare the results of the slab model with a "ground truth" result use `RADMC-3D`. The following cells are taken from the `radmc3d.ipynb` notebook but only creates the setup and computes the radio image.

In [ ]:
plt.rcParams["figure.dpi"] = 150.
rt = radmc3d.Model(data)


# temporary fix for old data file
rt.R_star_ = rt.R_star_[0]
rt.M_star_ = rt.M_star_[0]
rt.T_star_ = rt.T_star_[0]



rt.phii_grid = np.array([0., 2.*np.pi])
rt.ai_grid = np.geomspace(rt.a_dust_.min(), 1., 17)
rt.radmc3d_options["nphot"] = 500_000
rt.radmc3d_options["nphot_scat"] = 500_000
rt.radmc3d_options["nphot_spec"] = 10_000
rt.radmc3d_options["mc_scat_maxtauabs"] = 5.
rt.radmc3d_options["dust_2daniso_nphi"] = 60
rt.datadir = "radmc3d"
rt.write_files()
img_name = Path(rt.datadir) / 'image_radio.out'

Run these to compute/rename the image file if you want to re-recreate it

    !cd {rt.datadir} && radmc3d image lambda 1300 sizeau 100 npixx 512 npixy 512 setthreads 8
    !mv  {Path(rt.datadir) / 'image.out'} {img_name}

In [ ]:
image_radio = radmc3d.read_image(img_name)

In [ ]:
x, y = image_radio["x"] / c.au, image_radio["y"] / c.au

fig, ax = plt.subplots(figsize=(6, 6))
ax.set_aspect(1)
pcm = ax.pcolormesh(
    x, y, image_radio["I"][..., 0].T,
    cmap="inferno", vmin=0, vmax=image_radio['I'].max())
ax.set(xlabel=r"$X\,\left[\mathrm{au}\right]$", ylabel=r"$Y\,\left[\mathrm{au}\right]$")

pos = ax.get_position()
cbar = fig.colorbar(pcm, cax=fig.add_axes([1.02 * pos.x1, pos.y0, 0.02, pos.y1 - pos.y0]))
cbar.set_label(r"Intensity [erg/s/cm²/Hz/ster]")

In [ ]:
from dustpylib.radtrans import slab

In [ ]:
if rt.opacity == 'birnstiel2018':
    opacity_file = 'default_opacities_smooth.npz'
else:
    opacity_file = None

In [ ]:
from importlib import reload
from dustpylib.radtrans.slab import slab
reload(slab)

In [ ]:
opac = slab.Opacity(opacity_file)
lam = image_radio['lambda']

-----

In [ ]:
from scipy.interpolate import RegularGridInterpolator

In [ ]:
opac._lam.shape

In [ ]:
opac._a.shape

In [ ]:
opac._k_abs.shape

In [ ]:
lam = np.array([0., 1.])
a = np.linspace(1, 2, 1)
k_abs = np.arange(len(lam)*len(a)).reshape(len(a), len(lam))
#k_abs = np.array([3., 4.])

In [ ]:
_interp_k_abs = RegularGridInterpolator((lam, a), k_abs.T)

-----

In [ ]:
from scipy.stats import binned_statistic
X, Y = np.meshgrid(image_radio['x'], image_radio['y'], indexing='ij')
R = np.sqrt(X**2 + Y**2)
# define the bins
ri_bins = np.geomspace(np.abs(R).min(), R.max(), 100)
r_bins = 0.5 * (ri_bins[1:] + ri_bins[:-1])
# get the averaged intensity profile
profile = binned_statistic(R.ravel(), image_radio['I'][...,0].ravel(), bins=ri_bins).statistic

In [ ]:
f, ax = plt.subplots()
ax.loglog(data.grid.r / c.au, obs.I_nu[0, 0, :] * slab.jy_sas)
ax.loglog(r_bins / c.au, profile)
ax.set(ylim=[1e-15, 1e-11], xlim=[1, 200]);